# QPEP analysis

This notebook looks at the results from the qpep performance analysis in the opensand testbed.

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pickle, os
import re

In [13]:
file = open('qpep_results/iperf_results/iperf_50err.data','rb')
data = pickle.load(file)
file.close()
print(data['iperf_0.1mb']['sent_bps'])

[318044000.0, 3870330000.0, 2069210000.0, 6771160.0, 5654490.0, 5128020.0]


In [18]:
file = open('qpep_results/raw_results10000000.0.data','rb')
data = pickle.load(file)
file.close()
print(data['end'])

{'streams': [{'sender': {'socket': 4, 'start': 0, 'end': 88.7207, 'seconds': 88.7207, 'bytes': 11319016, 'bits_per_second': 1020640.0, 'retransmits': 0}, 'receiver': {'socket': 4, 'start': 0, 'end': 88.7207, 'seconds': 88.7207, 'bytes': 10132408, 'bits_per_second': 913646}}], 'sum_sent': {'start': 0, 'end': 88.7207, 'seconds': 88.7207, 'bytes': 11319016, 'bits_per_second': 1020640.0, 'retransmits': 0}, 'sum_received': {'start': 0, 'end': 88.7207, 'seconds': 88.7207, 'bytes': 10132408, 'bits_per_second': 913646}, 'cpu_utilization_percent': {'host_total': 0.555172, 'host_user': 0.204351, 'host_system': 0.350825, 'remote_total': 0.0097447, 'remote_user': 0.00546568, 'remote_system': 0.00427903}}


I want to plot throughput (i.e. sent bps) against: delays or plr (on two different plots). On the same plot, we have qpep and plain for the 3 different file sizes. Put confidence intervals around the datapoints of one of the file sizes. Repeat for different bottleneck bandwidths.

I should create a dictionnary with
```bash
bandwidths
|-plr
|--delay
|---filesize [10kbps,100kbps,10mbps]
```


In [26]:
file = open('../qpep/opensand-testbed/results/qpep_iperf_0.01plr_10000.0kbps_100ms_.data','rb')
data = pickle.load(file)
file.close()
print(data)

{'iperf_0.01mb': {'sent_bytes': [6553600, 7864320, 7864320, 7864320, 7864320], 'sent_bps': [368964000000.0, 552056000000.0, 405974000000.0, 395626000000.0, 431181000000.0], 'received_bytes': [212553, 341171, 491520, 503773, 372701], 'received_bps': [11966600000.0, 23949400000.0, 25373300000.0, 25343100000.0, 20434300000.0]}, 'iperf_0.1mb': {'sent_bytes': [6553600, 7864320, 7864320, 7864320, 7864320], 'sent_bps': [4574150000.0, 414259000000.0, 372190000000.0, 3898980000.0, 495090000000.0], 'received_bytes': [233057, 367810, 502547, 493972, 370249], 'received_bps': [162664000.0, 19374700000.0, 23783800000.0, 244902000.0, 23308600000.0]}, 'iperf_10.0mb': {'sent_bytes': [18350080, 18350080, 18350080, 18350080, 18350080], 'sent_bps': [17736800.0, 17776000.0, 17736100.0, 17733400.0, 17696500.0], 'received_bytes': [10262511, 10393591, 10393586, 10262514, 10393586], 'received_bps': [9919550.0, 10068500.0, 10045800.0, 9917620.0, 10023400.0]}}


In [37]:
def average_bps(file_path):
    try:
        data = np.loadtxt(file_path)
        # only take values after time is more than 100
        return np.average(data[data[:,0] > 100])
    except: 
        return 0

def get_error(file_path):
    match = re.search(r"_([.\d]*)plr_", file_path)
    if not match: 
        print(f"Error: Error not found in {file_path}")
        return None
    return float(match.group(1))/100

def get_delay(file_path):
    match = re.search(r"_(\d+?\.\d+?)ms_",file_path)
    if not match: 
        print(f"Error: Delay not found in {file_path}")
        return None
    return match.group(1)

def get_bandwidth(file_path):
    match = re.search(r"_(\d+?.\d+?kbps)_",file_path)
    if not match: 
        print(f"Error: Bandwidth not found in {file_path}")
        return None
    return match.group(1)

In [41]:
import matplotlib.colors as mcolors
error_percentages = [0,0.01,0.5,1] #that is in packet error
file_sizes = [1e4,1e5,1e7]#in bytes
bandwidths = [7e2,1e4,4e5]#in kbps
delays = [1,10,50,100,250]#Added delay on the satellite link (=RTT/4)

my_colours = list(mcolors.TABLEAU_COLORS.values())
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
cca = "Opensand emulation"
folder_path = "./qpep_results/18.08.23/qpep_iperf/"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)
# Create dictionary to store the average for each error, rtt, and run
results = {}
for i,filename in enumerate(file_list):
    bandwidth = get_bandwidth(f'{folder_path}{filename}')
    error = get_error(f'{folder_path}{filename}')
    rtt = get_delay(f'{folder_path}{filename}')
    if bandwidth not in results:
        results[bandwidth] = {}
    if error not in results[bandwidth]:
        results[bandwidth][error] = {}
    if rtt not in results[bandwidth][error]:
        results[bandwidth][error][rtt] = []
    file = open(f'{folder_path}{filename}','rb')
    data = pickle.load(file)
    for filesize in data.keys():
        print(data[filesize]['sent_bps'])
        results[bandwidth][error][rtt][filesize].append(data[filesize]['sent_bps'])
    file.close()

print(results)

# fig, ax = plt.subplots(1,1,figsize=(6,4))
# # sort the dictionary by error key
# results[bw] = dict(sorted(results[bw].items()))
# for j,error in enumerate(results[bw]):
#     points = np.zeros((len(results[bw][error]),4))
#     for i, rtt in enumerate(results[bw][error]):
#         points[i,:] = rtt , np.average(results[bw][error][rtt])/mtu, np.min(results[bw][error][rtt])/mtu, np.max(results[bw][error][rtt])/mtu
#     points = points[points[:,0].argsort()]
#     ax.errorbar(points[:,0],points[:,1], yerr=[abs(points[:,1]-points[:,2]), abs(points[:,3]-points[:,1])], fmt='none', capsize=5,alpha=0.5, color=my_colours[j%len(my_colours)])
#     #ax.errorbar(points[:,0],points[:,1], yerr=[points[:,1]*0.1, points[:,1]*0.08], fmt='none', capsize=5,alpha=0.5, color=my_colours[j%len(my_colours)])
#     ax.plot(points[:,0],points[:,1],label=error,marker='x', markerfacecolor="none", markersize = '4',linestyle='',color=my_colours[j%len(my_colours)])
# ax.plot(points[:,0],bdw*points[:,0]/1000/mtu,linestyle='--',color='black', alpha = 0.5,label="BDP")
# ax.set_ylim(0, 35)
# ax.text(0.40, 0.95, f"{cca}, {bw}", transform=ax.transAxes, fontsize="small",
#         verticalalignment='top', bbox=props)
# ax.legend(title="Error rate",loc='upper left',fontsize="small", markerscale=0.6, ncols=2)
# plt.tick_params(axis='both', direction='in',labelsize="small", grid_alpha=0.5, grid_linestyle=':')
# ax.grid()
# ax.set_xlabel("RTT (ms)")
# ax.set_ylabel("cwnd (packets)")
# plt.show()

Error: Delay not found in ./qpep_results/18.08.23/qpep_iperf/qpep_iperf_1plr_700.0kbps_50ms_.data
[951869000.0, 239894000.0, 238808000.0, 239425000.0, 239330000.0]


TypeError: list indices must be integers or slices, not str